In [1]:
# So we can import our modules 
import sys
sys.path.append('../..')

import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

from captum.attr import (
  visualization as viz,
  IntegratedGradients,
  LayerIntegratedGradients,
  DeepLift,
  LayerDeepLift,
  DeepLiftShap,
  LayerDeepLiftShap,
  GradientShap,
  LayerGradientShap,
  GuidedGradCam,
  LayerGradCam,
  LayerAttribution,
  LRP,
  LayerLRP,
  NoiseTunnel
)

from src.utils.data import load_split, get_transforms
from src.utils.model import init_model
from src.datasets.pornography_frame_dataset import PornographyFrameDataset

In [2]:
INPUT_SHAPE = 224
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

DATA_LOC = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/even-20"
STATE_DICTS_LOC = "../../results/even-20/data-aug/pornography-2k/models"

MODEL_NAME = "vgg19"
state_dict_path = f"{STATE_DICTS_LOC}/{MODEL_NAME}_freeze_False_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.pth"

In [3]:
df_test = load_split(DATA_LOC, [0.1, 0.2], ["test"])["test"]
data_transforms = get_transforms(False, INPUT_SHAPE, MEAN, STD)["test"]
dataset = PornographyFrameDataset(DATA_LOC, df_test, data_transforms)

Loaded split.
test: total (8000); porn (4020); non-porn (3980)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

state_dict = torch.load(state_dict_path, map_location=device)
model = init_model(MODEL_NAME)
model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
model.eval()

last_conv_layer = model.module.features[-3]

Device: cpu


: 

In [ ]:
idx = 4140
frame_name, frame_tensor, label = dataset[idx]
print(f"name (label): {frame_name} ({label})")

input = frame_tensor.unsqueeze(0)
input.requires_grad = True

output = model(input)
pred_confidence, pred_label = torch.max(torch.sigmoid(output), dim=1)
print(f"{MODEL_NAME} - ground-truth: {label} | predicted: {pred_label.squeeze().item()} ({pred_confidence.squeeze().item()})")

In [ ]:
model.module.features

## GradCAM

In [ ]:
grad_cam = LayerGradCam(model, last_conv_layer)
lgc_attr = grad_cam.attribute(input, target=pred_label)
lgc_attr_upsampled = LayerAttribution.interpolate(lgc_attr, input.shape[2:])

guided_grad_cam = GuidedGradCam(model, last_conv_layer)
gcc_attr = guided_grad_cam.attribute(input, target=pred_label)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,6))

viz.visualize_image_attr(
  lgc_attr_upsampled[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  cmap="jet",
  show_colorbar=True,
  title="GradCAM",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[0])
)

viz.visualize_image_attr(
  gcc_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  cmap="jet",
  show_colorbar=True,
  title="Guided GradCAM",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[1])
)

plt.show()

## Integrated Gradients

In [ ]:
integrated_gradients = IntegratedGradients(model)
ig_attr = integrated_gradients.attribute(input, target=pred_label, n_steps=10) # TODO: change number of steps

layer_integrated_gradients = LayerIntegratedGradients(model, model.module.features[-3])
lig_attr = layer_integrated_gradients.attribute(input, target=pred_label, n_steps=10)
lig_attr_upsampled = LayerAttribution.interpolate(lig_attr, input.shape[2:])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,12))

viz.visualize_image_attr(
  ig_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="Integrated Gradients",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[0])
)

viz.visualize_image_attr(
  lig_attr_upsampled[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="Integrated Gradients",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[1])
)

fig.show()

In [ ]:
noise_tunnel = NoiseTunnel(integrated_gradients)
ig_nt_attr = noise_tunnel.attribute(input, target=pred_label, nt_samples=10, nt_type="smoothgrad_sq")

In [ ]:
viz.visualize_image_attr(
  ig_nt_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  cmap="jet",
  show_colorbar=True,
  title="Noise Tunnel"
)

## DeepLIFT 

DeepLIFT is a backpropagation-based approach which takes a fundamental different perspective

In [ ]:
# TODO: understand baselines

# Definition of baseline distribution of images
rand_dist = torch.cat([input * 0, input * 1])

deep_lift = DeepLift(model) 
dl_attr = deep_lift.attribute(input, target=pred_label)

layer_deep_lift = LayerDeepLift(model, layer=last_conv_layer)
ldl_attr = layer_deep_lift.attribute(input, target=pred_label)
ldl_attr_upsampled = LayerAttribution.interpolate(ldl_attr, input.shape[2:])

deep_lift_shap = DeepLiftShap(model)
dls_attr = deep_lift_shap.attribute(input, baselines=rand_dist, target=pred_label)

layer_deep_lift_shap = LayerDeepLiftShap(model, layer=last_conv_layer)
ldls_attr = layer_deep_lift_shap.attribute(input, baselines=rand_dist, target=pred_label)
ldls_attr_upsampled = LayerAttribution.interpolate(ldls_attr, input.shape[2:])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12,12))

viz.visualize_image_attr(
  dl_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="DeepLIFT",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[0][0])
)

viz.visualize_image_attr(
  ldl_attr_upsampled[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="Layer DeepLIFT",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[0][1])
)

viz.visualize_image_attr(
  dls_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="DeepLIFT Shap",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[1][0])
)

viz.visualize_image_attr(
  ldls_attr_upsampled[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="Layer DeepLIFT Shap",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[1][1])
)

fig.show()

## Gradient Shap

In [ ]:
# Definition of baseline distribution of images
rand_dist = torch.cat([input * 0, input * 1])

gradient_shap = GradientShap(model)
gs_attr = gradient_shap.attribute(input, baselines=rand_dist, target=pred_label)

layer_gradient_shap = LayerGradientShap(model, layer=last_conv_layer)
lgs_attr = layer_gradient_shap.attribute(input, baselines=rand_dist, target=pred_label)
lgs_attr_upsampled = LayerAttribution.interpolate(lgs_attr, input.shape[2:])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,6))

viz.visualize_image_attr(
  gs_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="Gradient Shap",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[0])
)

viz.visualize_image_attr(
  lgs_attr_upsampled[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  show_colorbar=True,
  title="Layer Gradient Shap",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[1])
)

## LRP

In [ ]:
lrp = LRP(model)
lrp_attr = lrp.attribute(input, target=pred_label)

layer_lrp = LayerLRP(model, layer=last_conv_layer)
llrp_attr = layer_lrp.attribute(input, target=pred_label)
llrp_attr_upsampled = LayerAttribution.interpolate(llrp_attr, input.shape[2:])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,6))

viz.visualize_image_attr(
  lrp_attr[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  cmap="jet",
  show_colorbar=True,
  title="LRP",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[0])
)

viz.visualize_image_attr(
  llrp_attr_upsampled[0].cpu().permute(1,2,0).detach().numpy(),
  input.squeeze().cpu().permute(1,2,0).detach().numpy(),
  method="blended_heat_map",
  sign="positive",
  cmap="jet",
  show_colorbar=True,
  title="Layer LRP",
  use_pyplot=False,
  plt_fig_axis=(fig,axes[1])
)